In [1]:
import carla #the sim library itself
import cv2 #to work with images from cameras
import time # towork with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import random
import sys

In [2]:
PREFERRED_SPEED = 30 # what it says
SPEED_THRESHOLD = 2 
MAX_STEER_DEGREES = 40
STEERING_CONVERSION = 75

# maintain speed function
def maintain_speed(s):
    ''' 
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 

def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

# function to get angle between the car and target waypoint
def get_angle(car,wp):
    '''
    this function to find direction to selected waypoint
    '''
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    # vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y),(car_vector.x,car_vector.y))

    return degrees

# Function to calculate the Euclidean distance between two points
def calculate_distance(location1, location2):
    dx = location1.x - location2.x
    dy = location1.y - location2.y
    dz = location1.z - location2.z
    return math.sqrt(dx ** 2 + dy ** 2 + dz ** 2)

# Spawn a car behind and to the right of spawn_points[0]
def spawn_vehicle_behind_right(world, vehicle_bp, reference_point):
    # Create an offset to move the spawn point
    offset_distance = 5  # Adjust this value to place the car behind and to the right
    offset_behind = -5  # Place the car behind the original point
    offset_right = 3     # Place the car to the right of the original point
    
    # Adjust the spawn location using offset
    new_spawn_location = carla.Location(
        reference_point.location.x + offset_behind,
        reference_point.location.y + offset_right,
        reference_point.location.z
    )

    new_spawn_transform = carla.Transform(new_spawn_location, reference_point.rotation)
    
    # Spawn the vehicle at the new location
    vehicle = world.try_spawn_actor(vehicle_bp[0], new_spawn_transform)
    return vehicle

In [11]:
# connect to the sim 
client = carla.Client('localhost', 2000)

world = client.get_world()
spawn_points = world.get_map().get_spawn_points()
#look for a blueprint of Mini car

start_point = spawn_points[0]

vehicle_bp = world.get_blueprint_library().filter('*model3*')[0]
new_spawn_location = carla.Location(
    start_point.location.x+15,
    start_point.location.y-10,
    start_point.location.z
)
desired_rotation = carla.Rotation(pitch=0.0, yaw=90.0, roll=0.0)
new_spawn_transform = carla.Transform(new_spawn_location, desired_rotation)
vehicle = world.spawn_actor(vehicle_bp, new_spawn_transform)

# Get the original location of the vehicle
original_location = new_spawn_location
# print(start_point)

#main loop 
quit = False

while True:
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        quit = True
        break
    steering_angle = 0 
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    estimated_throttle = maintain_speed(speed)
    vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, 
                                               steer=steering_angle))
    
    # Get the current location of the vehicle
    current_location = vehicle.get_location()

    # Calculate the distance between the original spawn point and the current position
    distance = calculate_distance(current_location, original_location)
    print(current_location)
    print(distance)
    if distance >= 17:
        vehicle.destroy()
        vehicle = world.try_spawn_actor(vehicle_bp, new_spawn_transform)


Location(x=-49.644844, y=14.471010, z=0.597294)
0.002706289291381836
Location(x=-49.644844, y=14.471010, z=0.590883)
0.009116828441619873
Location(x=-49.644844, y=14.471010, z=0.584180)
0.01582014560699463
Location(x=-49.644844, y=14.471010, z=0.575790)
0.024210095405578613
Location(x=-49.644844, y=14.471010, z=0.560118)
0.03988158702850342
Location(x=-49.644844, y=14.471010, z=0.547570)
0.05243027210235596
Location(x=-49.644844, y=14.471010, z=0.534035)
0.06596499681472778
Location(x=-49.644844, y=14.471010, z=0.508992)
0.0910075306892395
Location(x=-49.644844, y=14.471010, z=0.491706)
0.10829430818557739
Location(x=-49.644844, y=14.471010, z=0.460885)
0.13911446928977966
Location(x=-49.644844, y=14.471010, z=0.438314)
0.16168612241744995
Location(x=-49.644844, y=14.471010, z=0.413509)
0.1864910125732422
Location(x=-49.644844, y=14.471010, z=0.374336)
0.22566422820091248
Location(x=-49.644844, y=14.471010, z=0.345744)
0.2542556822299957
Location(x=-49.644844, y=14.471010, z=0.315099)


KeyboardInterrupt: 

: 

In [10]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()